In [1]:
import convokit
from convokit import Corpus, download
from collections import defaultdict

In [2]:
corpus = Corpus(filename=download('reddit-corpus-small'))

Dataset already exists at C:\Users\rgang\.convokit\downloads\reddit-corpus-small


In [3]:
# speaker_to_convos = defaultdict(set)
# for convo in corpus.iter_conversations():
speaker_convo_utts = defaultdict(lambda: defaultdict(list))
for speaker in corpus.iter_objs('speaker'):
    for utt in speaker.iter_utterances():
        if utt.text:
            speaker_convo_utts[speaker.id][utt.conversation_id].append(utt.text)

In [4]:
speaker_convo_utts

defaultdict(<function __main__.<lambda>()>,
            {'AutoModerator': defaultdict(list,
                         {'9c716m': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9cfz10': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9d07z8': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9dbjxa': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9dnnwe': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9e7yw7': ['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n'],
                          '9egz42': ['

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
speaker_convo_models = defaultdict(dict)
speaker_docs = defaultdict(list)
for speaker,convos in speaker_convo_utts.items():
    
    for convo_id, utterances in convo.items():
        if speaker_convo_utts[speaker][convo]:
            try:
                speaker_convo_models[speaker][convo] = cv.fit_transform([' '.join(speaker_convo_utts[speaker][convo])])
            except ValueError:
                continue

In [10]:
speaker_convo_models

se matrix of type '<class 'numpy.int64'>'
              	with 33 stored elements in Compressed Sparse Row format>},
             'OldManPastry': {'9c6lxz': <1x19 sparse matrix of type '<class 'numpy.int64'>'
              	with 19 stored elements in Compressed Sparse Row format>,
              '9c3dn6': <1x13 sparse matrix of type '<class 'numpy.int64'>'
              	with 13 stored elements in Compressed Sparse Row format>},
             'JTCMuehlenkamp': {'9c6lxz': <1x4 sparse matrix of type '<class 'numpy.int64'>'
              	with 4 stored elements in Compressed Sparse Row format>,
              '9j2w1e': <1x5 sparse matrix of type '<class 'numpy.int64'>'
              	with 5 stored elements in Compressed Sparse Row format>},
             'LumberChaton': {'9c6lxz': <1x29 sparse matrix of type '<class 'numpy.int64'>'
              	with 29 stored elements in Compressed Sparse Row format>},
             'ilayabr': {'9c6lxz': <1x20 sparse matrix of type '<class 'numpy.int64'>'
   

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

speaker_convo_models2 = defaultdict(dict)
doc_indices = defaultdict(dict)
for speaker, convos in speaker_convo_utts.items():
    speaker_convos = []
    i = 0
    for convo_id, utterances in convos.items():
        if utterances:
            doc_indices[speaker][convo_id] = i
            speaker_convos.append(' '.join(utterances))
        i += 1
    try:
        cv = CountVectorizer()
        cv.fit(speaker_convos)
        speaker_convo_models2[speaker] = cv
    except ValueError:
        continue

In [48]:
for obj in corpus.iter_objs('utterance'):
    speaker_id = obj.get_speaker().get_id()
    convo_id = obj.get_conversation().get_id()
    speaker_model = speaker_convo_models2[speaker_id]
    print([' '.join(speaker_convo_utts[speaker_id][convo_id])])
    print([' '.join(y) for x,y in speaker_convo_utts[speaker_id].items() if x != convo_id])
    break

['Talk about your day. Anything goes, but subreddit rules still apply. Please be polite to each other! \n']


In [8]:
joined_text = {speaker: {convo: ' '.join(convo_utterances) for convo, convo_utterances in convos.items()} for speaker, convos in speaker_convo_utts.items()}

In [9]:
joined_text['bklynsnow']

{'9ccgtz': "I'm not your friend, pal. ",
 '9jfsl7': 'If I\'m in the mood and she\'s not, she will sometimes offer to "let me go".  \nI prefer that she be involved too, of course, but if she offers and I\'m desperate, I\'ll sometimes take her up on it.  \nI can\'t imagine just moving forward if she hasn\'t agreed to it. Who the fuck does that?',
 '9ckfnl': 'Perfect description. Lmao'}

In [38]:
x = speaker_convo_models2['bklynsnow'].transform([y for x,y in joined_text['bklynsnow'].items()])
y = speaker_convo_models2['bklynsnow'].transform([joined_text['bklynsnow']['9ckfnl']])

In [53]:
import numpy as np
x = np.asarray(x.sum(axis=0)).squeeze()
y = np.asarray(y.sum(axis=0)).squeeze()

In [54]:
N_context=x.sum()
N_target=y.sum()

In [46]:
np.asarray(np.log2(x.sum(axis=0)/N)).squeeze()

array([-5.88264305, -4.88264305, -5.88264305, -5.88264305, -5.88264305,
       -5.88264305, -5.88264305, -5.88264305, -5.88264305, -5.88264305,
       -5.88264305, -5.88264305, -5.88264305, -5.88264305, -5.88264305,
       -4.29768055, -5.88264305, -5.88264305, -5.88264305, -4.88264305,
       -5.88264305, -5.88264305, -5.88264305, -5.88264305, -5.88264305,
       -5.88264305, -5.88264305, -4.88264305, -5.88264305, -5.88264305,
       -5.88264305, -5.88264305, -5.88264305, -5.88264305, -5.88264305,
       -3.56071495, -4.88264305, -5.88264305, -4.88264305, -4.88264305,
       -4.88264305, -5.88264305, -5.88264305, -5.88264305, -5.88264305,
       -5.88264305])

In [55]:
y/N

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01694915, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01694915, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01694915, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [56]:
np.dot(np.log2(x/N_context), y/N_target)

-5.882643049361842